In [45]:
from matplotlib import pyplot as plt 
import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) 
# change file_data to where did you put it! 
file_data = 'D:\\Python\\MachineLearning\\MachineLearning\\week04\\data\\glass.csv' 
glass_df = pd.read_csv(file_data) 

print(glass_df.info()) 
glass_df.drop(columns='Id_Number', inplace=True)
glass_types = glass_df['Type'].unique() 
print(glass_types) 
print(glass_df['Type'].value_counts()) 
X = glass_df[glass_df.columns[:-1]] 
y = glass_df['Type'] 
y = np.asarray(y) - 1
X = np.asarray(X)
C = len(np.unique(y)) # number of classes (for c = 1, 2, 3, 4, 5, 6, 7)
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25, random_state=42)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Id_Number  214 non-null    int64  
 1   RI         214 non-null    float64
 2   Na         214 non-null    float64
 3   Mg         214 non-null    float64
 4   Al         214 non-null    float64
 5   Si         214 non-null    float64
 6   K          214 non-null    float64
 7   Ca         214 non-null    float64
 8   Ba         214 non-null    float64
 9   Fe         214 non-null    float64
 10  Type       214 non-null    int64  
dtypes: float64(9), int64(2)
memory usage: 18.5 KB
None
[1 2 3 5 6 7]
Type
2    76
1    70
7    29
3    17
5    13
6     9
Name: count, dtype: int64


In [46]:
from scipy import sparse  
def convert_labels(y, C = C): 
    """ 
    convert 1d label to a matrix label: each column of this  
    matrix coresponding to 1 element in y. In i-th column of Y,  
    only one non-zeros element located in the y[i]-th position,  
    and = 1 ex: y = [0, 2, 1, 0], and 3 classes then return 
 
            [[1, 0, 0, 1], 
             [0, 0, 1, 0], 
             [0, 1, 0, 0]] 
    """ 
    Y = sparse.coo_matrix((np.ones_like(y), (y, np.arange(len(y)))), shape = (C, len(y))).toarray() 
    return Y  
 
# Y = convert_labels(y, C) 

In [47]:
def softmax_stable(Z): 
    """
     Compute softmax values for each sets of scores in Z. 
    each column of Z is a set of score.     
    """ 
    e_Z = np.exp(Z - np.max(Z, axis = 0, keepdims = True)) 
    A = e_Z / e_Z.sum(axis = 0) 
    return A 
def softmax(Z): 
    """ 
    #Compute softmax values for each sets of scores in V. 
    #each column of V is a set of score.     
    """ 
    e_Z = np.exp(Z) 
    A = e_Z / e_Z.sum(axis = 0) 
    return A 
def softmax_regression(X, y, W_init, eta, tol = 1e-4, max_count = 10000): 
    W = [W_init]     
    C = W_init.shape[1] 
    Y = convert_labels(y, C) 
    it = 0 
    N = X.shape[1] 
    d = X.shape[0] 
     
    count = 0 
    check_w_after = 20 
    while count < max_count: 
        # mix data  
        mix_id = np.random.permutation(N) 
        for i in mix_id: 
            xi = X[:, i].reshape(d, 1) 
            yi = Y[:, i].reshape(C, 1) 
            ai = softmax(np.dot(W[-1].T, xi)) 
            W_new = W[-1] + eta*xi.dot((yi - ai).T) 
            count += 1 
            # stopping criteria 
            if count%check_w_after == 0:                 
                if np.linalg.norm(W_new - W[-check_w_after]) < tol: 
                    return W 
            W.append(W_new) 
    return W 
 
# cost or loss function   
def cost(X, Y, W): 
    A = softmax(W.T.dot(X)) 
    return -np.sum(Y*np.log(A)) 
 
# Predict that X belong to which class (1..C now indexed as 0..C-1 )  
def pred(W, X): 
    """ 
    predict output of each columns of X 
    Class of each x_i is determined by location of max probability 
    Note that class are indexed by [0, 1, 2, ...., C-1] 
    """ 
    A = softmax_stable(W.T.dot(X)) 
    return np.argmax(A, axis = 0) 
 
# W[-1] is the solution, W is all history of weights

In [49]:
print(X.shape)
print(y.shape)

eta = .05  
d = X.T.shape[0]
W_init = np.random.randn(X.T.shape[0], C) 
W = softmax_regression(X.T, y, W_init, eta) 
print(W[-1]) 

(214, 9)
(214,)


ValueError: The maximum label in y should be less than the number of classes C

Code bằng thư viện Scikit learn

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import sklearn 
#from sklearn.preprocessing import StandardScaler  
from sklearn.linear_model import LogisticRegression 

# Call to Logistic Regression Model - SAG: solving is based on Stochastic Average Gradient 
lorg=LogisticRegression(multi_class='multinomial',solver='sag', max_iter=5000) 
# and train model by Training Dataset 
lorg.fit(X_train,y_train)  
 
# Then Predict the Test data 
Y_pred=lorg.predict(X_test) 
 
# for accuracy 
from sklearn.metrics import accuracy_score 
print("Accuracy Score: ")
print(accuracy_score(y_test,Y_pred)) 
 
# for confusion matrix 
from sklearn.metrics import confusion_matrix 
cm=confusion_matrix(y_test,Y_pred) 
print("Confusion Matrix:")
print(cm) 
print("Coefficients:")
print(lorg.coef_)
y_predict = lorg.predict(X_test)
print("Predict:")
print(y_predict)

Accuracy Score: 
1.0
Confusion Matrix:
[[14  0  0  0  0  0]
 [ 0 21  0  0  0  0]
 [ 0  0  4  0  0  0]
 [ 0  0  0  4  0  0]
 [ 0  0  0  0  3  0]
 [ 0  0  0  0  0  8]]
Coefficients:
[[-0.75230967  0.02492735  0.17789738  0.05817981 -0.06762927  1.0950278
  -0.04380848  0.39046056  0.00884578 -0.00503627]
 [-0.29280393  0.01372743  0.09785311  0.03438748  0.15248986  0.73801103
   0.11451411 -0.20556688 -0.00978722  0.05082582]
 [ 0.0201071   0.0027883   0.16136776  0.53774044 -0.26999699  0.06705451
  -0.13486328 -0.07005493 -0.13231887 -0.02425521]
 [ 0.14084856  0.00350954 -0.42078776 -0.51067315  0.38132906 -0.12946595
   0.40668107  0.26924942  0.18633299  0.02268501]
 [ 0.28964919 -0.01968951  0.18299318 -0.0876475  -0.23361499 -0.55363428
  -0.33362644 -0.00809419 -0.19584034 -0.04067724]
 [ 0.59450875 -0.0252631  -0.19932366 -0.03198708  0.03742233 -1.2169931
  -0.00889697 -0.37599398  0.14276766 -0.00354211]]
Predict:
[1 7 1 7 2 2 1 2 2 2 6 5 2 2 6 5 7 1 1 6 2 7 7 7 3 2 1 1 5 1 1